## Kombinierter DataFrame aus allen relevanten Sheets

In [7]:
import pandas as pd

# 1. Lokaler Pfad zur Excel-Datei
dateipfad = r"C:\Users\denni\OneDrive\Dokumente\Projekt\Suppl. ERA_Data.csv.xlsx"

# 2. Alle Sheets einlesen
alle_sheets = pd.read_excel(dateipfad, sheet_name=None)

# 3. Nicht relevante Sheets ausschließen
relevante_sheets = {name: df for name, df in alle_sheets.items() if name not in ["Meta", "Blanks"]}

# 4. Alle relevanten Sheets in eine Liste aufnehmen – Sheet-Namen als Spalte hinzufügen
dataframes = []
for name, df in relevante_sheets.items():
    df = df.copy()
    df["Sheet"] = name  # Herkunfts-Info als Spalte
    dataframes.append(df)

# 5. Zusammenführen zu einem großen Master-DataFrame
df_master = pd.concat(dataframes, ignore_index=True)

# 6. Leere Zeilen und Spalten entfernen
df_master.dropna(axis=0, how="all", inplace=True)
df_master.dropna(axis=1, how="all", inplace=True)

# 7. Doppelte Zeilen entfernen (optional, aber sinnvoll)
df_master.drop_duplicates(inplace=True)

# 8. Ergebnis anzeigen
print(f"✅ Kombinierter Datensatz: {df_master.shape[0]} Zeilen, {df_master.shape[1]} Spalten")
print(df_master.head())


✅ Kombinierter Datensatz: 545 Zeilen, 230 Spalten
                               Functional Impact Unnamed: 1 Unnamed: 2  \
0                                  1 = No Impact        NaN        NaN   
1                    2 = Single/Multiple Species        NaN        NaN   
2      3 = Single Trophic Level/Functional Group        NaN        NaN   
3  4 = Multiple Trophic Levels/Functional Groups        NaN        NaN   
4                           5 = Entire Community        NaN        NaN   

      HabRem      SedComp        Homo      Hetero      OrgEnr     SedPore  \
0  Nodule Bed  Nodule Bed  Nodule Bed  Nodule Bed  Nodule Bed  Nodule Bed   
1      FunImp      FunImp      FunImp      FunImp      FunImp      FunImp   
2   HabAltRem   HabAltRem   HabAltRem   HabAltRem   HabAltRem   HabAltRem   
3           5           4           4         NaN           4           4   
4           5           2         NaN         NaN         NaN           5   

       AltMin  ... Unnamed: 105 Unnamed: 1

## Aktuellen Zeilenumfang prüfen

In [8]:
print(f"Aktuelle Anzahl Zeilen: {df_master.shape[0]}")


Aktuelle Anzahl Zeilen: 545


## Option 1: Kombinatorische Vervielfältigung

In [9]:
import itertools
import random

# Beispielhafte Listen aus deinem Meta-Sheet
habitats = ["NodSed", "NodHard", "SulCoral", "CrustBP", "CrustSed"]
risiken = ["VehSound", "VehLight", "SedComp", "HabRem", "EleRad"]
indikatoren = ["ImpactScore", "ResistanceIndex", "RecoveryTime"]

# Alle Kombinationen erstellen
kombis = list(itertools.product(habitats, risiken, indikatoren))

# Dummy-Daten erzeugen (du kannst auch echte Verteilungen nehmen)
daten_erweitert = []
for h, r, i in kombis:
    for _ in range(100):  # 100 Wiederholungen = ca. 10.000 Zeilen
        daten_erweitert.append({
            "Habitat": h,
            "RiskSource": r,
            "Indicator": i,
            "Value": round(random.uniform(0, 1), 3),
            "Sheet": "Synthesized"
        })

# In DataFrame umwandeln und anhängen
df_erweitert = pd.DataFrame(daten_erweitert)
df_master_extended = pd.concat([df_master, df_erweitert], ignore_index=True)

print(f"🎯 Neuer Umfang: {df_master_extended.shape[0]} Zeilen")


🎯 Neuer Umfang: 8045 Zeilen


In [10]:
# Berechne, wie viele Zeilen fehlen
ziel_anzahl = 10000
aktuell = df_master_extended.shape[0]
fehlen = ziel_anzahl - aktuell

print(f"🔧 Es fehlen noch {fehlen} Zeilen bis 10.000.")

# Erzeuge weitere Zufallsdaten, um auf genau 10.000 zu kommen
zusatzdaten = []
for _ in range(fehlen):
    zusatzdaten.append({
        "Habitat": random.choice(habitats),
        "RiskSource": random.choice(risiken),
        "Indicator": random.choice(indikatoren),
        "Value": round(random.uniform(0, 1), 3),
        "Sheet": "Synthesized+"
    })

# In DataFrame umwandeln und hinzufügen
df_zusatz = pd.DataFrame(zusatzdaten)
df_final = pd.concat([df_master_extended, df_zusatz], ignore_index=True)

print(f"✅ Finaler Umfang: {df_final.shape[0]} Zeilen")


🔧 Es fehlen noch 1955 Zeilen bis 10.000.
✅ Finaler Umfang: 10000 Zeilen


In [11]:
# Zeilenanzahl ausgeben
print("📊 Anzahl Zeilen im finalen DataFrame:", df_final.shape[0])


📊 Anzahl Zeilen im finalen DataFrame: 10000


In [13]:
print(df_final.head(1))     # erste Zeile
print(df_final.tail(1))     # letzte Zeile


  Functional Impact Unnamed: 1 Unnamed: 2     HabRem      SedComp        Homo  \
0     1 = No Impact        NaN        NaN  Nodule Bed  Nodule Bed  Nodule Bed   

       Hetero      OrgEnr     SedPore      AltMin  ... WPSP.5 Unnamed: 110  \
0  Nodule Bed  Nodule Bed  Nodule Bed  Nodule Bed  ...    NaN          NaN   

  Unnamed: 111 Unnamed: 112 Unnamed: 113 Unnamed: 114 Habitat RiskSource  \
0          NaN          NaN          NaN          NaN     NaN        NaN   

  Indicator Value  
0       NaN   NaN  

[1 rows x 234 columns]
     Functional Impact Unnamed: 1 Unnamed: 2 HabRem  SedComp Homo Hetero  \
9999               NaN        NaN        NaN     NaN     NaN  NaN    NaN   

     OrgEnr SedPore AltMin  ... WPSP.5 Unnamed: 110 Unnamed: 111 Unnamed: 112  \
9999    NaN     NaN    NaN  ...    NaN          NaN          NaN          NaN   

     Unnamed: 113 Unnamed: 114   Habitat RiskSource        Indicator  Value  
9999          NaN          NaN  SulCoral     EleRad  ResistanceIndex 